In [7]:
import os
import sys

# 🔴 MANUALLY SET PROJECT ROOT (ONCE)
PROJECT_ROOT = r"C:\Users\keshav bansal\PycharmProjects\PythonProject2\Satellite-Imagery-Based-Property-Valuation"

# Force working directory
os.chdir(PROJECT_ROOT)

# Add to import path
sys.path.append(PROJECT_ROOT)

print("Project root set to:", PROJECT_ROOT)
print("Working directory:", os.getcwd())
import os
import sys
import numpy as np

import torch
from torch.utils.data import DataLoader

from src.datasets import MultimodalDataset
from src.models import MultimodalRegressor

from sklearn.metrics import mean_squared_error, r2_score


Project root set to: C:\Users\keshav bansal\PycharmProjects\PythonProject2\Satellite-Imagery-Based-Property-Valuation
Working directory: C:\Users\keshav bansal\PycharmProjects\PythonProject2\Satellite-Imagery-Based-Property-Valuation
ERROR! Session/line number was not unique in database. History logging moved to new session 68


In [8]:
features = [
    "bedrooms", "bathrooms", "sqft_living_capped", "sqft_lot",
    "floors", "waterfront", "view", "condition", "grade",
    "lat", "long", "house_age", "bathroom_per_bedroom"
]


In [9]:
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("data/processed/train_processed.csv")

train_idx, val_idx = train_test_split(
    df.index,
    test_size=0.2,
    random_state=42
)

train_ds = MultimodalDataset(
    csv_path="data/processed/train_processed.csv",
    image_dir="images/train",
    features=features,
    target="log_price",
    indices=train_idx
)

val_ds = MultimodalDataset(
    csv_path="data/processed/train_processed.csv",
    image_dir="images/train",
    features=features,
    target="log_price",
    indices=val_idx
)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)



MultimodalDataset: 12961 valid samples out of 16209
MultimodalDataset: 3240 valid samples out of 16209


In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = MultimodalRegressor(tabular_dim=len(features)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = torch.nn.MSELoss()
for param in model.image_encoder.parameters():
    param.requires_grad = False

C:\Users\keshav bansal\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\keshav bansal\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, tabular, y in train_loader:
        images = images.to(device)
        tabular = tabular.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        preds = model(images, tabular)
        loss = loss_fn(preds, y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1, Loss: 8.2051
Epoch 2, Loss: 0.9316
Epoch 3, Loss: 0.6352
Epoch 4, Loss: 0.4466
Epoch 5, Loss: 0.3185


In [12]:
model.eval()

y_true, y_pred = [], []

with torch.no_grad():
    for images, tabular, y in val_loader:
        images = images.to(device)
        tabular = tabular.to(device)

        preds = model(images, tabular)

        y_true.extend(y.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print("Validation RMSE (log):", rmse)
print("Validation R² (log):", r2)

Validation RMSE (log): 0.5947853693206145
Validation R² (log): -0.28190815201804753
